# English - Alice in wonderland

## Data preparation

In [1]:
from importlib import reload

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/pavlem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:
import preprocessor
preprocessor = reload(preprocessor)


pp = preprocessor.Preprocessor("data/original/alice_in_wonderland.txt", "eng", False)
cleaned_corpus = pp.process()
pp.split(cleaned_corpus)

[later editions continued as follows When the sands are all dry, he is gay as a lark, And will talk in contemptuous tones of the Shark, But, when the tide rises and sharks are around, His voice has a timid and tremulous sound.].
[later editions continued as follows The Panther took pie-crust, and gravy, and meat, While the Owl had the dish as its share of the treat.
Number of unique characters: 35
Number of characters: 367
dict_keys(['[', 'l', 'a', 't', 'e', 'r', ' ', 'd', 'i', 'o', 'n', 's', 'c', 'u', 'f', 'w', 'W', 'h', 'y', ',', 'g', 'k', 'A', 'm', 'p', 'S', 'B', 'H', 'v', '.', ']', 'T', 'P', '-', 'O'])


## Subword segmentation

In [5]:
import sentpiece
sentpiece = reload(sentpiece)

# Character level train vocabular size = 68
# Character level test vocabular size = 68


NUM_CHARS = 2700
TRAIN_DATA_PATH = "data/processed/eng_test.txt"
MODEL_NAME = "eng_model_lrg"
TYPE = "_test"


sentpiece.train_model(TRAIN_DATA_PATH, MODEL_NAME + TYPE, NUM_CHARS)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/processed/eng_test.txt
  input_format: 
  model_prefix: eng_model_lrg_test
  model_type: BPE
  vocab_size: 2700
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespac

In [6]:
!mv $MODEL_NAME* spm_models/

In [7]:
SEG_DATA_FILE = "eng_lrg_test.txt"


sentpiece.segmentation(TRAIN_DATA_PATH, MODEL_NAME + TYPE, SEG_DATA_FILE)

## Train LM

In [8]:
TRAIN_SEG_DATA_PATH = "data/segmented/eng_lrg_train.txt"
TEST_SEG_DATA_PATH = "data/segmented/eng_lrg_test.txt"
NUM_LAYERS = 40
CLASS = 9999
BPTT = 3


!bash train_script.sh $TRAIN_SEG_DATA_PATH $TEST_SEG_DATA_PATH $NUM_LAYERS $CLASS $BPTT

rm: cannot remove 'rnnlm/models/model': No such file or directory
rm: cannot remove 'rnnlm/models/model.output.txt': No such file or directory
debug mode: 2
train file: data/segmented/eng_lrg_train.txt
valid file: data/segmented/eng_lrg_test.txt
class size: 9999
Hidden layer size: 40
BPTT: 3
Rand seed: 1
rnnlm file: model
Starting training using file data/segmented/eng_lrg_train.txt
Vocab size: 2334
Words in train file: 29151
Iter:   0	Alpha: 0.100000	   TRAIN entropy: 9.0951    Words/sec: 839.9   VALID entropy: 8.3108
Iter:   1	Alpha: 0.100000	   TRAIN entropy: 8.1197    Words/sec: 885.8   VALID entropy: 7.8704
Iter:   2	Alpha: 0.100000	   TRAIN entropy: 7.6205    Words/sec: 946.2   VALID entropy: 7.7488
Iter:   3	Alpha: 0.100000	   TRAIN entropy: 7.2727    Words/sec: 962.1   VALID entropy: 7.5793
Iter:   4	Alpha: 0.100000	   TRAIN entropy: 7.0079    Words/sec: 982.4   VALID entropy: 7.3153
Iter:   5	Alpha: 0.100000	   TRAIN entropy: 6.7887    Words/sec: 984.7   VALID entropy: 7.1944


In [9]:
MODEL_OUTPUT = MODEL_NAME + ".output.txt"

!mv model $MODEL_NAME
!mv model.output.txt $MODEL_OUTPUT

!mv $MODEL_NAME $MODEL_OUTPUT rnnlm_models/

## Data generation

In [52]:
GEN_DATA_PATH = "data/generated/eng_model_chr/"
MODEL_PATH = f"./rnnlm_models/{MODEL_NAME}"

!bash gen_script.sh $GEN_DATA_PATH $MODEL_PATH

In [56]:
# Decoding generated texts

for i in range(1, 8):
    size = 10**i
    sentpiece.desegmentation(f"data/generated/eng_model_chr/{size}.txt", MODEL_NAME + TYPE)

# Bengali

# Analysis